In [393]:
import requests
from xml.etree import ElementTree
from collections import defaultdict
import numpy as np
import pandas as pd
import statistics
import math
import seaborn as sns
import matplotlib.pyplot as plt
import ast
import json
import urllib.request
import escher
from escher import Builder
import cobra
import time
from time import sleep

In [394]:
def etree_to_dict(t):
    d = {t.tag: {} if t.attrib else None}
    children = list(t)
    if children:
        dd = defaultdict(list)
        for dc in map(etree_to_dict, children):
            for k, v in dc.items():
                dd[k].append(v)
        d = {t.tag: {k: v[0] if len(v) == 1 else v
                     for k, v in dd.items()}}
    if t.attrib:
        d[t.tag].update(('@' + k, v)
                        for k, v in t.attrib.items())
    if t.text:
        text = t.text.strip()
        if children or t.attrib:
            if text:
                d[t.tag]['#text'] = text
        else:
            d[t.tag] = text
    return d

In [395]:
def createTree(link):
    time.sleep(1)
    response = requests.get(link)
    tree = ElementTree.fromstring(response.content)
    return tree

In [396]:
def createDictFromName(complexList, errors):
    result = {}

    for name in complexList:
        try:
            linkComplex = "https://websvc.biocyc.org/getxml?id=ECOLI:" + name
            treeComplex = createTree(linkComplex)
            treeDictComplex = etree_to_dict(treeComplex)
            result[name] = treeDictComplex
        except Exception as e:
            errors.append((name, e))

    return result

In [397]:
def createDictFromLink(complexList, errors):
    result = {}

    for name in complexList:
        try:
            linkComplex = "https://websvc.biocyc.org/" + name
            treeComplex = createTree(linkComplex)
            treeDictComplex = etree_to_dict(treeComplex)
            result[name] = treeDictComplex
        except Exception as e:
            errors.append((name, e))

    return result

In [398]:
simData = np.load(r"../../out/geneRxnVerifData/output.npy", allow_pickle=True, encoding='ASCII')
fluxesWithCaption = simData.tolist()['agents']['0']['listeners']['fba_results']['estimated_fluxes']
complexes = simData.tolist()['agents']['0']['bulk']

metabolData = np.load(r"../../out/geneRxnVerifData/stoichiometry.npy", allow_pickle=True, encoding='ASCII')
rxn_metabolites = metabolData.tolist()

In [399]:
reactions = fluxesWithCaption.keys()

In [400]:
ecReactions = pd.read_csv('reactions.tsv', sep='\t', header=0)['Reaction'].tolist()

In [401]:
foundNames = []
notFound = []
for rxn in ecReactions:
    found = False
    for key in fluxesWithCaption:
        if rxn in key:
            found = True
            foundNames.append((rxn, key))
    if not found:
        notFound.append(rxn)

In [402]:
prev = ""
fluxes = {}
for names in foundNames:
    if prev == names[0]:
        for i in range(len(fluxes[prev])):
            if "(reverse)" in names[1]:
                fluxes[prev][i] -= fluxesWithCaption[names[1]][i]
            else:
                fluxes[prev][i] += fluxesWithCaption[names[1]][i]
    else:
        prev = names[0]
        fluxes[prev] = fluxesWithCaption[names[1]]

In [403]:
with open('../../iJO1366.Central metabolism.json') as json_file:
    data = json.load(json_file)
    
escherRxns = data[1]['reactions']
escherNames = []

for number in escherRxns:
    reactionName = escherRxns[number]['name']
    escherNames.append(reactionName.lower())

In [404]:
errors = []
reactionDict = createDictFromName(fluxes.keys(), errors)

In [405]:
%store reactionDict

Stored 'reactionDict' (dict)


In [406]:
ECnames = {}

for key in reactionDict.keys():
    dicte = reactionDict[key]
    names = []
    
    try:
        temp = dicte['ptools-xml']['Reaction']['enzymatic-reaction']['Enzymatic-Reaction']
        
        if type(temp) is list:
            for item in temp:
                if item['common-name']['#text'] not in names:
                    names.append(item['common-name']['#text'])
        else:
            names.append(dicte['ptools-xml']['Reaction']['enzymatic-reaction']['Enzymatic-Reaction']['common-name']['#text'])
            
    except Exception as e1:
        try:
            nest = dicte['ptools-xml']['Reaction']['in-pathway']['Reaction']['@resource']
            temp = []
            temp.append(nest)
            new = createDictFromLink(temp, errors)
            temp1 = new[temp[0]]['ptools-xml']['Reaction']['enzymatic-reaction']['Enzymatic-Reaction']
            
            if type(temp1) is list:
                for item in temp1:
                    if item['common-name']['#text'] not in names:
                        names.append(item['common-name']['#text'])
            else:
                names.append(temp['common-name']['#text'])
                
        except Exception as e2:
            errors.append((key, e1, e2))
            
    try:
        temp = dicte['ptools-xml']['Reaction']['enzymatic-reaction']['Enzymatic-Reaction']
        
        if type(temp) is list:
            for item in temp:
                names += [x['#text'] for x in item['synonym']]
        else:
            names += [x['#text'] for x in temp['synonym']]
            
    except Exception as e1:
        try:
            temp = dicte['ptools-xml']['Reaction']['enzymatic-reaction']['Enzymatic-Reaction']
        
            if type(temp) is list:
                for item in temp:
                    names.append(item['synonym']['#text'])
            else:
                names.append(item['synonym']['#text'])
        except Exception as e2:
            errors.append((key, e1, e2))
        
    ECnames[key] = names
    

In [407]:
foundNames = {}

for key in ECnames.keys():
    names1 = ECnames[key]
    for name in names1:
        if name.lower() in escherNames:
            foundNames[key] = name[0].upper() + name[1:]
            break

In [408]:
foundNames

{'1PFRUCTPHOSN-RXN': 'Phosphofructokinase',
 'GLYCEROL-1-PHOSPHATASE-RXN': 'Glycerol-3-phosphatase',
 '3PGAREARR-RXN': 'Phosphoglycerate mutase',
 '1TRANSKETO-RXN': 'Transketolase',
 'TRANSALDOL-RXN': 'Transaldolase',
 'GLUC1PADENYLTRANS-RXN': 'Glucose-1-phosphate adenylyltransferase',
 '2PGADEHYDRAT-RXN': 'Enolase',
 'MALATE-DEHYDROGENASE-ACCEPTOR-RXN': 'Malate dehydrogenase',
 'PGLUCISOM-RXN': 'Glucose-6-phosphate isomerase',
 'RXN-15513': 'Phosphoglycerate mutase',
 'CITSYN-RXN': 'Citrate synthase',
 'RIBOKIN-RXN': 'Ribokinase',
 'DLACTDEHYDROGNAD-RXN': 'D-lactate dehydrogenase',
 'METHGLYSYN-RXN': 'Methylglyoxal synthase',
 'CATAL-RXN': 'Catalase',
 'R601-RXN': 'Fumarate reductase',
 '2TRANSKETO-RXN': 'Transketolase',
 'GAPOXNPHOSPHN-RXN': 'Glyceraldehyde-3-phosphate dehydrogenase',
 'PYRUVDEH-RXN': 'Pyruvate dehydrogenase',
 'GLYCOLATEDEHYDRO-RXN': 'Glycolate oxidase',
 'XYLULOKIN-RXN': 'Xylulokinase',
 '6PGLUCONOLACT-RXN': '6-phosphogluconolactonase',
 'GLUCOSE-1-PHOSPHAT-RXN': '

In [409]:
newFluxes = {}

for key in fluxes.keys():
    if key in foundNames.keys():
        newFluxes[foundNames[key]] = fluxes[key][-1]

In [417]:
additional = {'D-glucose transport via PEP:Pyr PTS': 0,
 'FRUpts2': 0,
 'Glucose-6-phosphate isomerase': -3665600,
 'Phosphofructokinase': 0,
 'Fructose-bisphosphatase': 38308,
 'Fructose-bisphosphate aldolase': 385734,
 'Triose-phosphate isomerase': 107701,
 'Glyceraldehyde-3-phosphate dehydrogenase': 177895,
 'Phosphoglycerate kinase': -711572,
 'Phosphoglycerate mutase': 0,
 'Phosphoglycerate mutase': -708632,
 'Enolase': 177268,
 'Pyruvate kinase': 0,
 'Phosphoenolpyruvate synthase': 445971}

In [418]:
additional2 = {'D-glucose transport via PEP:Pyr PTS': 516124,
 'FRUpts2': 0,
 'Glucose-6-phosphate isomerase': 395206,
 'Phosphofructokinase': 90275,
 'Fructose-bisphosphatase': 9774,
 'Fructose-bisphosphate aldolase': 151536,
 'Triose-phosphate isomerase': -491270,
 'Glyceraldehyde-3-phosphate dehydrogenase': 517023,
 'Phosphoglycerate kinase': -1034042,
 'Phosphoglycerate mutase': 0,
 'Phosphoglycerate mutase': -1032508,
 'Enolase': 516362,
 'Pyruvate kinase': 0,
 'Phosphoenolpyruvate synthase': 0}

In [425]:
for x in additional2.keys():
    newFluxes[x] = additional2[x]

In [426]:
newFluxes

{'Phosphofructokinase': 90275,
 'Glycerol-3-phosphatase': 0,
 'Phosphoglycerate mutase': -1032508,
 'Transketolase': 24877,
 'Transaldolase': -128192,
 'Glucose-1-phosphate adenylyltransferase': 9983,
 'Enolase': 516362,
 'Malate dehydrogenase': -383976,
 'Glucose-6-phosphate isomerase': 395206,
 'Citrate synthase': 30326,
 'Ribokinase': 0,
 'D-lactate dehydrogenase': 0,
 'Methylglyoxal synthase': 187,
 'Catalase': 70,
 'Fumarate reductase': 22352,
 'Glyceraldehyde-3-phosphate dehydrogenase': 517023,
 'Pyruvate dehydrogenase': 0,
 'Glycolate oxidase': 0,
 'Xylulokinase': 0,
 '6-phosphogluconolactonase': 159683,
 'Glucose-1-phosphatase': 753,
 'Isocitrate lyase': -22785,
 'Acetaldehyde dehydrogenase (acetylating)': 221,
 'Citrate lyase': 0,
 'Phosphoenolpyruvate carboxylase': 225749,
 'Phosphotransacetylase': -17487,
 'Pyruvate oxidase': 0,
 'Triose-phosphate isomerase': -491270,
 'Ribose-5-phosphate isomerase': -90935,
 'Glycerol dehydrogenase': -12473,
 'Glucose-6-phosphate phosphatas

In [427]:
escher.list_available_maps()

[{'organism': 'Saccharomyces cerevisiae',
  'map_name': 'iMM904.Central carbon metabolism'},
 {'organism': 'Homo sapiens',
  'map_name': 'RECON1.Inositol retinol metabolism'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Glycolysis TCA PPP'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Tryptophan metabolism'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Carbohydrate metabolism'},
 {'organism': 'Homo sapiens',
  'map_name': 'RECON1.Amino acid metabolism (partial)'},
 {'organism': 'Escherichia coli', 'map_name': 'iJO1366.Nucleotide metabolism'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Fatty acid biosynthesis (saturated)'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Nucleotide and histidine biosynthesis'},
 {'organism': 'Escherichia coli', 'map_name': 'e_coli_core.Core metabolism'},
 {'organism': 'Escherichia coli', 'map_name': 'iJO1366.Central metabolism'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Fatty acid beta-oxidation'}

In [428]:
escher.list_available_models()

[{'organism': 'Saccharomyces cerevisiae', 'model_name': 'iMM904'},
 {'organism': 'Homo sapiens', 'model_name': 'RECON1'},
 {'organism': 'Escherichia coli', 'model_name': 'e_coli_core'},
 {'organism': 'Escherichia coli', 'model_name': 'iJO1366'}]

In [429]:
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    model_name='e_coli_core',
    reaction_data={r: v for r, v in zip(newFluxes.keys(), newFluxes.values())},
)

In [430]:
builder

Builder(reaction_data={'Phosphofructokinase': 90275, 'Glycerol-3-phosphatase': 0, 'Phosphoglycerate mutase': -…

In [347]:
ECnames['PGLUCISOM-RXN']

['glucose-6-phosphate isomerase',
 'phosphoglucose isomerase',
 'D-glucose-6-phosphate-ketol-isomerase']

In [331]:
reactionDict['PGLUCISOM-RXN']['ptools-xml']['Reaction']['enzymatic-reaction']['Enzymatic-Reaction'][0]['common-name']['#text']

'glucose-6-phosphate isomerase'

In [332]:
reactionDict['PGLUCISOM-RXN']['ptools-xml']['Reaction']['enzymatic-reaction']['Enzymatic-Reaction'][1]['common-name']['#text']

'glucose-6-phosphate isomerase'

In [335]:
reactionDict['PGLUCISOM-RXN']['ptools-xml']['Reaction']['enzymatic-reaction']['Enzymatic-Reaction'][0]['synonym']

[{'@datatype': 'string', '#text': 'phosphoglucose isomerase'},
 {'@datatype': 'string', '#text': 'D-glucose-6-phosphate-ketol-isomerase'}]

In [336]:
def getNames(liste, names):
    for item in liste:
        if item['common-name']['#text'] not in names:
            names.append(item['common-name']['#text'])
    for item in liste:
        names
        

SyntaxError: unexpected EOF while parsing (29049002.py, line 2)

In [431]:
np.save("glucoseReactionDict.npy", reactionDict)